In [1]:
import numpy as np
import pandas as pd
import time, os, sys, random, re, itertools
import matplotlib as mpl
import PIL.Image
import skimage.io as imgio
import skimage.filters as skf
import skimage.exposure as ske
import scipy.ndimage as ndi
from scipy import stats
import colorcet as cc
import pims
import skimage.filters
from IPython.display import clear_output
import xml.etree.ElementTree as ET
PIL.Image.MAX_IMAGE_PIXELS = None

In [2]:
eDir = ""
imgDir = "raw/"
maskDir = "mask/"
ssTabFile = eDir + "slideset-table.xml"

ssTab = ET.parse(ssTabFile)
imageFiles = [e.text for e in ssTab.find("./col[@name='Img']").findall("e")]
imageNames = list(map(lambda f : os.path.basename(f), imageFiles))
try:
    excludeMaskFiles = [e.text for e in ssTab.find("./SlideSet/col[@name='Mask image']").findall("e")]
    excludeMaskDict = dict(zip(imageNames, excludeMaskFiles))
except:
    excludeMaskDict = dict(itertools.zip_longest(imageNames, "x", fillvalue="x"))

jpgDir = "jpg/"

In [3]:
def relLimitsImage(I, l, h, excludeZeros=True):
    I = I.astype(float)
    if excludeZeros: I[I == 0] = np.nan
    [valLow, valHigh] = np.nanquantile(I, [l, h])
    return np.fmax(np.nan_to_num((I - valLow)/(valHigh - valLow)), 0) # NaN=0, Set floor to 0

#######
# Function for belnding with alpha channel (RGB + RGBA)
def blendWithAlpha(U, V):
    return (U * np.expand_dims(1-V[:,:,3],2)) + (V[:,:,0:3] * np.expand_dims(V[:,:,3],2))  

def visualizeMe(imageName):
    clear_output(wait=True)
    print("Now working on: " + imageName)
    
    # Set up file names
    imgShortName = re.findall(r'batchName-\d_\d+', imageName)[0]
    imgFile = eDir + imgDir + imageName
    maskFile = eDir + maskDir + imageName + "_masks-400.png"
    excludeFile = eDir + excludeMaskDict[imageName]
    plaFile = eDir + "plaDetect/" + imgShortName + "_pointMask.png"
    
    # Load and pre-process the image (may need to adjust this depending on image format)
    I = pims.Bioformats(imgFile)
    I.bundle_axes = 'yxc'
    img = I[0]
    masks = imgio.imread(maskFile)
    plaPointMask = imgio.imread(plaFile)
    try:
        excludeMask = imgio.imread(excludeFile)
    except:
        excludeMask = np.zeros_like(masks)
        
    # Make the data frame for this image
    df = pd.DataFrame({
        "idx": np.ravel(masks, order='C'),
        "K5": np.ravel(img[:,:,1], order='C'),
        "K6": np.ravel(img[:,:,2], order='C'),
        "PLA": np.ravel(img[:,:,3], order='C'),
        "PLApointMask": np.ravel(plaPointMask, order='C'),
        "exclude": np.ravel(excludeMask, order='C')
    })
    df = df.groupby("idx", as_index=False).agg(
        size=("PLApointMask", np.size),
        K5_med=("K5", "median"),
        K6_med=("K6", "median"),
        PLA_med=("PLA", "median"),
        PLA_points=("PLApointMask", "sum"),
        exclude=("exclude", np.max)
    )
    df["K5_med_scale"] = (df["K5_med"] - np.nanmin(df["K5_med"])) / (np.nanmax(df["K5_med"]) - np.nanmin(df["K5_med"]))
    df["K6_med_scale"] = (df["K6_med"] - np.nanmin(df["K6_med"])) / (np.nanmax(df["K6_med"]) - np.nanmin(df["K6_med"]))
    df["img"] = imgShortName
    df["group"] = df["img"].str.extract(r'((?<=batchName-)\d)')
    df["Krt"] = "?"
    df.loc[np.logical_and(df["K6_med"]<0.75*df["K5_med"], df["K5_med"]>2000), "Krt"] = "K5"
    df.loc[np.logical_and(df["K6_med"]>1.25*df["K5_med"], df["K6_med"]>2000), "Krt"] = "K6"
    df.loc[np.logical_and(df["K5_med"]<=2000, df["K6_med"]<=2000), "Krt"] = "wt"
    df["PLA_density"] = df["PLA_points"] / df["size"]
        
    # Create contours
    contours = np.zeros_like(masks)
    struct_elem = ndi.generate_binary_structure(2, 1)
    thick_struct_elem = ndi.iterate_structure(
        struct_elem, 4
    ).astype(bool)
    contours = (
        ndi.grey_dilation(masks, footprint=thick_struct_elem) !=
        ndi.grey_erosion(masks, footprint=thick_struct_elem)
    )
    
    # Create Krt mask
    KrtMask = np.expand_dims(np.zeros_like(masks), 2)
    KrtMask = np.where(np.expand_dims(np.array(df["Krt"]=="K5")[masks], 2), [[[0,1,0,0.75]]], KrtMask)
    KrtMask = np.where(np.expand_dims(np.array(df["Krt"]=="K6")[masks], 2), [[[1,0,0,0.75]]], KrtMask)
    KrtMask = np.where(np.expand_dims(np.array(df["Krt"]=="wt")[masks], 2), [[[0,0,1,0.75]]], KrtMask)
    KrtMask = np.where(np.expand_dims(np.array(df["Krt"]=="?")[masks], 2), [[[1,1,0,0.75]]], KrtMask)
    KrtMask = np.where(np.expand_dims(np.array(df["exclude"]>0)[masks], 2), KrtMask*[[[1,1,1,0.5]]], KrtMask)
    
    # Create PLA mask
    plaLabelMask = np.zeros_like(masks)
    plaLabelMask = ndi.binary_dilation(plaPointMask, iterations=5)
    plaLabelMask = plaLabelMask != ndi.binary_erosion(plaLabelMask, iterations=2)    
    
    #######
    # Show merge with mask outlines
    outFile = eDir + jpgDir + imageName + "_merge.jpg"
    U = np.expand_dims(relLimitsImage(img[:,:,0], 0.3, 0.98), 2) * [0,0.5,1] * 0.5
    U = U + ( np.expand_dims(relLimitsImage(img[:,:,1], 0.3, 0.98), 2) * [0,1,0.5] * 0.5 )
    U = U + ( np.expand_dims(relLimitsImage(img[:,:,2], 0.3, 0.98), 2) * [1,0.5,0] * 0.5 )
    U = U + ( np.expand_dims(relLimitsImage(img[:,:,3], 0.5, 0.999)**2, 2) * [1,0,0.5] )
    U = blendWithAlpha(U, np.where(np.expand_dims(contours,2), KrtMask, [[[0,0,0,0]]]))
    U = blendWithAlpha(U, np.where(np.expand_dims(plaLabelMask,2), [[[1,1,1,1]]], [[[0,0,0,0]]]))
    imgio.imsave(outFile, np.clip(U, 0, 1))
    
    #######
    # Show PLA density score with mask outlines
    outFile = eDir + jpgDir + imageName + "_PLA_density.png"
    magmaMap = mpl.cm.ScalarMappable(norm=None, cmap=mpl.cm.magma)
    group = re.search(r'((?<=batchName-)\d)', imgShortName).group(0)
    scale = 0.55
    U = magmaMap.to_rgba(np.array(df["PLA_density"])[masks]/scale, norm=False)[:,:,0:3]
    U = np.where(np.expand_dims(np.array(df["exclude"]>0)[masks], 2), [[[0,0,0]]], U)
    U = blendWithAlpha(U, np.where(np.expand_dims(contours,2), KrtMask, [[[0,0,0,0]]]))
    U[masks==0] = [[[0,0,0]]]
    imgio.imsave(outFile, np.clip(U, 0, 1))
    

In [ ]:
for i in imageNames: visualizeMe(i)